In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=1

# Create Data

In [2]:
import sys; sys.path.append("../../scripts/")
from generate_X import gen_trinmf_data
import matplotlib.pyplot as plt

kwkh=(5,3)
shape=(20,20)
data = gen_trinmf_data(shape=shape, 
                       kwkh=kwkh, 
                       factor_wh=(0.5, 1.0), 
                       factor_S=1,
                       random_state=10)
data.keys()

dict_keys(['X', 'W', 'S', 'H'])

In [3]:
X = data["X"]
Wtrue = data["W"]
Strue = data["S"]
Htrue = data["H"]
X.shape, Wtrue.shape, Strue.shape, Htrue.shape

((20, 20), (20, 5), (5, 3), (3, 20))

In [4]:
import numpy as np
Wcor= np.corrcoef(Wtrue.T)
maxcor = np.max(np.abs(Wcor[np.triu_indices(kwkh[0],k=2)]))
print(f'maximum correlation = {maxcor:.2f}')

maximum correlation = 0.39


# Initilize the Model

In [5]:
nmfk_params = {
    "n_perturbs":12,
    "n_iters":100,
    "epsilon":0.01,
    "n_jobs":-1,
    "init":"nnsvd",
    "use_gpu":False,
    "save_path":"../../results/",
    "verbose":True,
    "sill_thresh":0.8,
    "nmf_method":"nmf_fro_mu", 
    "perturb_type":"uniform", 
    "calculate_error":True,
    "pruned":True,
    "predict_k":True,
    "predict_k_method":"sill",
    "transpose":False,
    "mask":None,
    "use_consensus_stopping":False,
    "calculate_pac":True,
    "consensus_mat":True,
    "simple_plot":True,
    "collect_output":True
}

tri_nmfk_params = {
    "experiment_name":"TriNMFk",
    "nmfk_params":nmfk_params,
    "save_path":"../../results/",
    "nmf_verbose":False,
    "use_gpu":True,
    "n_jobs":-1,
    "mask":None,
    "use_consensus_stopping":False,
    "alpha":(0,0),
    "n_iters":100,
    "n_inits":10,
    "pruned":False,
    "transpose":False,
    "verbose":True
}

In [6]:
from TELF.factorization import TriNMFk
model = TriNMFk(**tri_nmfk_params)

n_perturbs: 20
perturb_type: uniform
n_iters: 100
epsilon: 0.015
init: nnsvd
save_path: ./
save_output: True
use_gpu: False
verbose: True
nmf_verbose: False
perturb_verbose: False
transpose: False
collect_output: False
sill_thresh: 0.8
predict_k: False
predict_k_method: pvalue
n_jobs: 1
n_nodes: 1
nmf: <function nmf at 0x135a80c20>
nmf_method: nmf_fro_mu
nmf_obj_params: {}
pruned: True
calculate_error: True
consensus_mat: True
use_consensus_stopping: 0
mask: None
calculate_pac: True
simple_plot: True
get_plot_data: False
perturb_multiprocessing: False
lock: <unlocked _thread.lock object at 0x1347b5c00>
total_exec_seconds: 0
experiment_name: 
nmf_params: {'niter': 100, 'use_gpu': False, 'nmf_verbose': False, 'mask': None, 'use_consensus_stopping': 0}


/Users/maksim/miniconda3/envs/TELF_public/lib/python3.11/site-packages/TELF-0.0.4-py3.11.egg/TELF/factorization/utilities/organize_n_jobs.py:10: UserWarning: No GPU found! Using CPUs
/Users/maksim/miniconda3/envs/TELF_public/lib/python3.11/site-packages/TELF-0.0.4-py3.11.egg/TELF/factorization/NMFk.py:587: UserWarning: calculate_error is True! Error calculation can make the runtime longer and take up more memory space!
/Users/maksim/miniconda3/envs/TELF_public/lib/python3.11/site-packages/TELF-0.0.4-py3.11.egg/TELF/factorization/utilities/organize_n_jobs.py:10: UserWarning: No GPU found! Using CPUs


# Perform NMFk First

In [7]:
Ks = range(1,7,1)
note = "This the the NMFk portion of the TriNMFk method!"
results = model.fit_nmfk(X, Ks, note)

  0%|          | 0/6 [00:00<?, ?it/s]

/Users/maksim/miniconda3/envs/TELF_public/lib/python3.11/site-packages/numpy-1.26.2-py3.11-macosx-11.1-arm64.egg/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/maksim/miniconda3/envs/TELF_public/lib/python3.11/site-packages/numpy-1.26.2-py3.11-macosx-11.1-arm64.egg/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/maksim/miniconda3/envs/TELF_public/lib/python3.11/site-packages/numpy-1.26.2-py3.11-macosx-11.1-arm64.egg/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/maksim/miniconda3/envs/TELF_public/lib/python3.11/site-packages/TELF-0.0.4-py3.11.egg/TELF/factorization/NMFk.py:368: RuntimeWarning: [tELF]: Encountered unknown logging metric "pac"
100%|██████████| 6/6 [00:00<00:00, 10.67it/s]


In [8]:
results.keys()

dict_keys(['time', 'k_predict'])

# Here now look at the outputs and choose k1 and k2 manually

In [10]:
! ls ../../results/TriNMFk_12perts_100iters_0.01eps_nnsvd-init

FINAL_k=1-6.png            WH_k=5.npz
WH_k=1.npz                 WH_k=6.npz
WH_k=2.npz                 experiment.log
WH_k=3.npz                 k_1_6_cophenetic_coeff.png
WH_k=4.npz                 k_4_con_mat.png


# Perform TriNMFk with the selected k1 and k2

In [7]:
k1k2=(5,3)
tri_nmfk_results = model.fit_tri_nmfk(X, k1k2)

100%|██████████| 10/10 [00:03<00:00,  2.55it/s]


In [12]:
tri_nmfk_results.keys()

dict_keys(['W', 'S', 'H', 'errors'])

In [13]:
W = tri_nmfk_results["W"]
S = tri_nmfk_results["S"]
H = tri_nmfk_results["H"]

In [14]:
W.shape, S.shape, H.shape

((20, 5), (5, 3), (3, 20))